# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.  
 
Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
# your code here
import pandas as pd
import re
import nltk
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm 

In [2]:
url = 'https://www.kaggle.com/datasets/kazanova/sentiment140'

In [3]:
response = requests.get(url)
print(response.status_code)
soup = BeautifulSoup(response.content, "html.parser")
soup

200



<!DOCTYPE html>

<html lang="en">
<head>
<title>Sentiment140 dataset with 1.6 million tweets | Kaggle</title>
<meta charset="utf-8"/>
<meta content="index, follow" name="robots"/>
<meta content="Sentiment analysis with tweets" name="description"/>
<meta content="no-cache" name="turbolinks-cache-control"/>
<meta content="internet,online communities,social networks,linguistics,languages" name="keywords"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=5.0, minimum-scale=1.0" name="viewport"/>
<meta content="#008ABC" name="theme-color">
<script nonce="GpPhwX4/E337PCNOSKRthg==" type="text/javascript">
    window["pageRequestStartTime"] = 1675330178854;
    window["pageRequestEndTime"] = 1675330178966;
    window["initialPageLoadStartTime"] = new Date().getTime();
  </script>
<link crossorigin="anonymous" href="https://www.google-analytics.com" rel="preconnect"/><link href="https://stats.g.doubleclick.net" rel="preconnect"/><link href="https://storage.googleapis.com" r

In [4]:
# target = []
# id = []
# text = []
# date = []
# flag = []
# user = []


In [5]:
raw_data = pd.read_csv('../Sentiment140.csv')
raw_data



,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


sampling to 20 k the data

In [6]:
raw_data_20k = raw_data.sample(n = 20000, random_state= 88)

display(raw_data_20k)
print('\/*\/*\/' * 30)

raw_data_10 = raw_data.sample(n = 10, random_state= 88)
raw_data_10

,target,ids,date,flag,user,text
209760,0,1974058893,Sat May 30 12:21:30 PDT 2009,NO_QUERY,BrookeAmanda,Ok it's only been a couple hours since dad has...
1259240,4,1998068077,Mon Jun 01 17:56:23 PDT 2009,NO_QUERY,KarinaKornacka,@graceofrhythm HAHA no i would never do that!!...
1266850,4,1999729993,Mon Jun 01 20:43:12 PDT 2009,NO_QUERY,stevegaghagen,Law of Attraction Creations: Law of Attraction...
1300432,4,2006627206,Tue Jun 02 11:26:46 PDT 2009,NO_QUERY,Hecie,is ordering ticketsssss EEEE (: &lt;3
1228994,4,1991292674,Mon Jun 01 06:46:16 PDT 2009,NO_QUERY,shanaloren,@STO_MAC nah im not mad at u....luv u too
...,...,...,...,...,...,...
560781,0,2205268283,Wed Jun 17 04:21:37 PDT 2009,NO_QUERY,neilina,"Today, not in a mood to do work"
125957,0,1834420698,Mon May 18 02:52:04 PDT 2009,NO_QUERY,Dirk_Gently,Thanks everyone for the birthday wishes! I rea...
879548,4,1685464907,Sun May 03 00:30:34 PDT 2009,NO_QUERY,jwalsh,@MFlanders but Vancouver will still steal your...
1556789,4,2185445238,Mon Jun 15 17:23:33 PDT 2009,NO_QUERY,hippychick_,"@keli_h great pic! So, I'm not the only one t..."


\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/


,target,ids,date,flag,user,text
209760,0,1974058893,Sat May 30 12:21:30 PDT 2009,NO_QUERY,BrookeAmanda,Ok it's only been a couple hours since dad has...
1259240,4,1998068077,Mon Jun 01 17:56:23 PDT 2009,NO_QUERY,KarinaKornacka,@graceofrhythm HAHA no i would never do that!!...
1266850,4,1999729993,Mon Jun 01 20:43:12 PDT 2009,NO_QUERY,stevegaghagen,Law of Attraction Creations: Law of Attraction...
1300432,4,2006627206,Tue Jun 02 11:26:46 PDT 2009,NO_QUERY,Hecie,is ordering ticketsssss EEEE (: &lt;3
1228994,4,1991292674,Mon Jun 01 06:46:16 PDT 2009,NO_QUERY,shanaloren,@STO_MAC nah im not mad at u....luv u too
972728,4,1832655595,Sun May 17 21:12:28 PDT 2009,NO_QUERY,GioWC,"@ century room, taste the rainbow with your fa..."
1554112,4,2184786636,Mon Jun 15 16:25:11 PDT 2009,NO_QUERY,LauraVogel,Ari also got a service award for the community...
636233,0,2233854669,Thu Jun 18 22:00:41 PDT 2009,NO_QUERY,Bugzbunny20,man I thought somethin was fina go done she di...
1042659,4,1957228583,Thu May 28 23:53:14 PDT 2009,NO_QUERY,ghbetbeze,@JaneBegger Leaving Moscow when it finally get...
123515,0,1833924004,Mon May 18 00:57:36 PDT 2009,NO_QUERY,htmlrulezd00d19,I wish I got a summer break!


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [7]:
# your code here
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tqdm import tqdm



def one_go_clean(text):
    ps = PorterStemmer()
    lem = WordNetLemmatizer() 
    stop_words = stopwords.words('english')
        
    pat = re.sub(r'https?://\S+', '', text) # remove the URL
    pat1 = re.sub('\W+', ' ', pat)  # remove Any non-alphanumeric character 
    s = re.sub('\d+', '', pat1).lower()  

    l = word_tokenize(s)  # tokenise the words
    

    stem_text = [ps.stem(w) for w in l]
    y = [lem.lemmatize(word) for word in stem_text]


    cleanned = [word for word in y if not word in stop_words]

    return cleanned

In [8]:
raw_data_10['text_processed'] = tqdm(raw_data_10['text'].apply(one_go_clean))
raw_data_10

100%|██████████| 10/10 [00:00<00:00, 21204.77it/s]


,target,ids,date,flag,user,text,text_processed
209760,0,1974058893,Sat May 30 12:21:30 PDT 2009,NO_QUERY,BrookeAmanda,Ok it's only been a couple hours since dad has...,"[ok, onli, coupl, hour, sinc, dad, ha, said, g..."
1259240,4,1998068077,Mon Jun 01 17:56:23 PDT 2009,NO_QUERY,KarinaKornacka,@graceofrhythm HAHA no i would never do that!!...,"[graceofrhythm, haha, would, never, actual, ma..."
1266850,4,1999729993,Mon Jun 01 20:43:12 PDT 2009,NO_QUERY,stevegaghagen,Law of Attraction Creations: Law of Attraction...,"[law, attract, creation, law, attract, creatio..."
1300432,4,2006627206,Tue Jun 02 11:26:46 PDT 2009,NO_QUERY,Hecie,is ordering ticketsssss EEEE (: &lt;3,"[order, ticket, eeee, lt]"
1228994,4,1991292674,Mon Jun 01 06:46:16 PDT 2009,NO_QUERY,shanaloren,@STO_MAC nah im not mad at u....luv u too,"[sto_mac, nah, im, mad, u, luv, u]"
972728,4,1832655595,Sun May 17 21:12:28 PDT 2009,NO_QUERY,GioWC,"@ century room, taste the rainbow with your fa...","[centuri, room, tast, rainbow, favorit, photog..."
1554112,4,2184786636,Mon Jun 15 16:25:11 PDT 2009,NO_QUERY,LauraVogel,Ari also got a service award for the community...,"[ari, also, got, servic, award, commun, servic..."
636233,0,2233854669,Thu Jun 18 22:00:41 PDT 2009,NO_QUERY,Bugzbunny20,man I thought somethin was fina go done she di...,"[man, thought, somethin, wa, fina, go, done, e..."
1042659,4,1957228583,Thu May 28 23:53:14 PDT 2009,NO_QUERY,ghbetbeze,@JaneBegger Leaving Moscow when it finally get...,"[janebegg, leav, moscow, final, get, warm]"
123515,0,1833924004,Mon May 18 00:57:36 PDT 2009,NO_QUERY,htmlrulezd00d19,I wish I got a summer break!,"[wish, got, summer, break]"


In [9]:
raw_data_20k['text_procesed'] = raw_data_20k['text'].apply(one_go_clean)
print("\n Only words in the text_processed:")
print('\/*\/*\/' * 30)
display(raw_data_20k)


 Only words in the text_processed:
\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/\/*\/*\/


,target,ids,date,flag,user,text,text_procesed
209760,0,1974058893,Sat May 30 12:21:30 PDT 2009,NO_QUERY,BrookeAmanda,Ok it's only been a couple hours since dad has...,"[ok, onli, coupl, hour, sinc, dad, ha, said, g..."
1259240,4,1998068077,Mon Jun 01 17:56:23 PDT 2009,NO_QUERY,KarinaKornacka,@graceofrhythm HAHA no i would never do that!!...,"[graceofrhythm, haha, would, never, actual, ma..."
1266850,4,1999729993,Mon Jun 01 20:43:12 PDT 2009,NO_QUERY,stevegaghagen,Law of Attraction Creations: Law of Attraction...,"[law, attract, creation, law, attract, creatio..."
1300432,4,2006627206,Tue Jun 02 11:26:46 PDT 2009,NO_QUERY,Hecie,is ordering ticketsssss EEEE (: &lt;3,"[order, ticket, eeee, lt]"
1228994,4,1991292674,Mon Jun 01 06:46:16 PDT 2009,NO_QUERY,shanaloren,@STO_MAC nah im not mad at u....luv u too,"[sto_mac, nah, im, mad, u, luv, u]"
...,...,...,...,...,...,...,...
560781,0,2205268283,Wed Jun 17 04:21:37 PDT 2009,NO_QUERY,neilina,"Today, not in a mood to do work","[today, mood, work]"
125957,0,1834420698,Mon May 18 02:52:04 PDT 2009,NO_QUERY,Dirk_Gently,Thanks everyone for the birthday wishes! I rea...,"[thank, everyon, birthday, wish, realli, hope,..."
879548,4,1685464907,Sun May 03 00:30:34 PDT 2009,NO_QUERY,jwalsh,@MFlanders but Vancouver will still steal your...,"[mflander, vancouv, still, steal, job, matter,..."
1556789,4,2185445238,Mon Jun 15 17:23:33 PDT 2009,NO_QUERY,hippychick_,"@keli_h great pic! So, I'm not the only one t...","[keli_h, great, pic, onli, one, read, girl, tub]"


In [10]:
from nltk.corpus import stopwords

def re_blob(row):
    return " ".join(row['text_procesed'])

raw_data_20k['clean_blob'] = raw_data_20k.apply(re_blob,axis=1)
raw_data_20k.head()

,target,ids,date,flag,user,text,text_procesed,clean_blob
209760,0,1974058893,Sat May 30 12:21:30 PDT 2009,NO_QUERY,BrookeAmanda,Ok it's only been a couple hours since dad has...,"[ok, onli, coupl, hour, sinc, dad, ha, said, g...",ok onli coupl hour sinc dad ha said go alreadi...
1259240,4,1998068077,Mon Jun 01 17:56:23 PDT 2009,NO_QUERY,KarinaKornacka,@graceofrhythm HAHA no i would never do that!!...,"[graceofrhythm, haha, would, never, actual, ma...",graceofrhythm haha would never actual made sus...
1266850,4,1999729993,Mon Jun 01 20:43:12 PDT 2009,NO_QUERY,stevegaghagen,Law of Attraction Creations: Law of Attraction...,"[law, attract, creation, law, attract, creatio...",law attract creation law attract creation today
1300432,4,2006627206,Tue Jun 02 11:26:46 PDT 2009,NO_QUERY,Hecie,is ordering ticketsssss EEEE (: &lt;3,"[order, ticket, eeee, lt]",order ticket eeee lt
1228994,4,1991292674,Mon Jun 01 06:46:16 PDT 2009,NO_QUERY,shanaloren,@STO_MAC nah im not mad at u....luv u too,"[sto_mac, nah, im, mad, u, luv, u]",sto_mac nah im mad u luv u


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [26]:
# your code here
import nltk 
from nltk import FreqDist

# List of list of all words
list_of_words = raw_data_20k["text_procesed"].tolist()
list_of_words

# Create one list of all words
all_words = []
for lists in list_of_words:
    for word in lists:
        all_words.append(word)

freq = FreqDist(raw_data_20k['clean_blob'])

top_5k_words = freq.most_common(5000)  # list of the 5k most frequents words
print(len(top_5k_words))
# print(top_5k_words)

5000


In [24]:
# separating list of words and frequency of words
word_features, word_freq = [[x for x,y in top_5k_words],
      [y for x,y in top_5k_words]]

# List of just words
print(len(word_features))
print(len(word_freq))
# print(freq_w)

5000
5000


### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [27]:
# your code here
def find_features(lst, all_words):
    word_features = list(all_words)
    words = set(lst)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

word_features
features=[]
for i,l in enumerate(raw_data_20k["text_procesed"]):
    s = [find_features(l,all_words), raw_data_20k["target"].iloc[i]]
    z = tuple(s)
    features.append(z)
len(features)

20000

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [1]:
# your code here

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [ ]:
# your code here

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here